In [ ]:
import requests,urllib3, logging, re, tqdm

from selenium import webdriver
import bs4 as bs
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.remote.remote_connection import LOGGER as seleniumLogger
from selenium.webdriver.chrome.service import Service
from urllib3.connectionpool import log as urllibLogger
from selenium.webdriver.common.action_chains import ActionChains

seleniumLogger.setLevel(logging.WARNING)
urllibLogger.setLevel(logging.WARNING)

import unicodedata
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
import time

In [1]:
#
#          'são paulo' : 'https://www.tripadvisor.com.br/Restaurants-g303631-Sao_Paulo_State_of_Sao_Paulo.html',
#          #NE
#          'recife' : 'https://www.tripadvisor.com.br/Restaurants-g304560-Recife_State_of_Pernambuco.html',
#          'fortaleza': 'https://www.tripadvisor.com.br/Restaurants-g303293-Fortaleza_State_of_Ceara.html',
#          'maceió': 'https://www.tripadvisor.com.br/Restaurants-g303216-Maceio_State_of_Alagoas.html',
#          'salvador': 'https://www.tripadvisor.com.br/Restaurants-g303272-Salvador_State_of_Bahia.html',
#          'joão pessoa': 'https://www.tripadvisor.com.br/Restaurants-g303428-Joao_Pessoa_State_of_Paraiba.html',
#          'natal': 'https://www.tripadvisor.com.br/Restaurants-g303518-Natal_State_of_Rio_Grande_do_Norte.html',
#          'aracaju': 'https://www.tripadvisor.com.br/Restaurants-g303638-Aracaju_State_of_Sergipe.html',
#          'são luiz': 'https://www.tripadvisor.com.br/Restaurants-g673267-Sao_Luis_State_of_Maranhao.html'
#        }

In [ ]:
class TripAdvisorScrap:
    
    def __init__(self, url):
        self.url = url
        self.links = {}
        self.root = 'https://www.tripadvisor.com.br/'
        self.acceptButton = '//*[@id="onetrust-accept-btn-handler"]'
        self.nextButton = 'a.nav.next.rndBtn.ui_button.primary.taLnk '
        self.css_selector_titles = 'a.Lwqic.Cj.b'
        self.timePause = 20
        self.driver = None

    def launch_browser(self):
        
        self.driver = webdriver.Chrome(ChromeDriverManager().install())
        self.driver.get(self.url)
        self.driver.implicitly_wait(2*self.timePause) 
        self.driver.find_element("xpath", self.acceptButton).click()
        return self.driver
    
  ### This needs the current driver state to get the correct title elements
  
    def getTitlesOneIter(self):
        html = bs.BeautifulSoup(self.driver.page_source, 'html.parser')
        for element in html.find_all('div', {'class':'tkvCJ u F f Ff K'}):
            self.links[element.a.text] = self.root + element.a['href'] 
        return 
    
    def pressNextButton(self):
        try:
            webNextButton = self.driver.find_element(By.CSS_SELECTOR, self.nextButton)
            self.driver.implicitly_wait(self.timePause) 
            ActionChains(self.driver).move_to_element(webNextButton).click().perform()
        except:
            pass
        return 
  #### n_iterations < n_pages, since not always the pressNextButton is effective in doing its job,
  #### therefore recquiring further iterations. Perhaps its wise to use above the beautiful soup option instead.
  #### Nevertheless, this is sufficient to proceed and to control the flow of the extraction.
          
    def getLinks(self, n_iterations):
        #start = time.time()
        if (self.driver==None):
            self.driver = self.launch_browser()
        else:
            self.driver = self.driver
        
        for n in tqdm.tqdm(range(0,n_iterations)):
            self.driver.implicitly_wait(2*self.timePause) 
            self.getTitlesOneIter()
            try:
                self.driver.implicitly_wait(3*self.timePause) 
                self.pressNextButton()
            except:
                pass
    
    
    #### Function to be used within the driveToLink. To open the hours tab, extending the possibility of HTML extraction.

        
    #### linkName : restaurant name. Index of self.links
   

In [ ]:
urla = 'https://www.tripadvisor.com.br/Restaurants-g304560-Recife_State_of_Pernambuco.html'

rec = TripAdvisorScrap(urla)

In [ ]:
rec.getLinks(5)

In [ ]:
rec.links

In [ ]:
rec.getLinks(80)

In [ ]:
rec.getLinks(60)

In [ ]:
len(rec.links)

In [ ]:
class ScrapLink():
    
    def __init__(self):
        self.root = 'https://www.tripadvisor.com.br/'
        self.acceptButton = '//*[@id="onetrust-accept-btn-handler"]'
        self.siteMenuClass = {'class' : 'YnKZo Ci Wc _S C AYHFM'}
        self.emailClass = {'class' : 'IdiaP Me sNsFa'}
        self.hourTabSelector = 'div.NehmB'
        self.hoursCssSelector = 'div.RiEuX.f'
        self.ifVerified = 'div.XAnbq._S'
        self.externalNearLinksClass = {'class' : "all-link"}
        self.foodCuisineMealsClass = {'class': "SrqKb"}
        self.addressClass = {'class': "kDZhm IdiaP Me"}
        self.commentsClass = {'class': "entry"}
        self.reviewsIdiomsAndClassificationClass = {'class': "choices"}
        self.datesOfReviewsClass = {'class':"prw_rup prw_reviews_stay_date_hsx"}
        self.bubbleRatingClass = {'class': 'YDAvY R2 F1 e k'}
        self.bubbleRatingInnerClass = {'class': "vzATR"}
        self.tripAdvRatingClass = {"class":"ZDEqb"}
        self.telephoneClass = {'class':'yEWoV'}
        self.nameClass = {'class':'HjBfq'}
        self.labelPriceClass = {'class':"dlMOJ"}
        self.tripNumberClass = {'class':'AfQtZ'}
        self.AnswerTable = {}
        self.exceptions_names = []
        self.exceptions = []
        self.html = None
        self.timePause = 25
        self.driver = webdriver.Chrome(ChromeDriverManager().install())    
    
    
    def scrapName(self):
        try:
            return self.html.find('h1', self.nameClass).text
        except:
            return None
        
    def scrapEmail(self):
        try:
            divEmail = self.html.find_all('div', self.emailClass)[1]
            Email = divEmail.a['href'].split('mailto:')[1].split('?subject=?')[0]
        except: 
            Email = None
        return Email
    
    def scrapNumber(self):
        try:
            return self.html.find('div', self.bubbleRatingClass).b.text
        except:
            return None
        
    def scrapLabelPrice(self):
        try:
            return self.html.find('a', self.labelPriceClass).text
        except:
            return None
    def scrapNumberReviews(self):
        try:
            return self.html.find('span', self.tripNumberClass).text
        except: 
            return None
    
    def scrapTelephone(self):
        try:
            return self.html.find('div',{'class':"IdiaP Me"}).find('a', href=True)['href']
        except:
            return None
    
        
    def scrapIfVerified(self):
        try:
            return self.driver.find_elements(By.CSS_SELECTOR, self.ifVerified)[0].text
        except:
            return None
    
    def scrapLinksMainAndMenuWebPages(self):
        SiteMenu =  self.html.find_all('a', self.siteMenuClass)
        try:
            Site = SiteMenu[0]["href"]
        except:
            Site = None
        try:
            Menu = divSiteMenu[1]["href"]
        except:
            Menu = None
        return Site, Menu
        
    def scrapNearExternalLinks(self):
        try:
            hotelsNear = self.root + self.html.find_all('a', self.externalNearLinksClass)[0]["href"]
        except:
            hotelsNear = None
        try:
            restaurantsNear = self.root + self.html.find_all('a', self.externalNearLinksClass)[1]["href"]
        except:
            restaurantsNear = None
        try:
            attractionsNear = self.root + self.html.find_all('a', self.externalNearLinksClass)[2]["href"]
        except:
            attractionsNear = None
            
        return hotelsNear, restaurantsNear, attractionsNear
            
    def scrapHorarios(self):
        try:
            horarios = self.driver.find_elements(By.CSS_SELECTOR, self.hoursCssSelector)        
            return [unicodedata.normalize("NFKD", horario.text).replace('\n',' - ') for horario in horarios]
        except:
            return None
            
    def scrapPriceAndFoodAndMeals(self):
        try: 
            priceFoodMeals = self.html.find_all('div', self.foodCuisineMealsClass)
            return [unicodedata.normalize("NFKD", pfm.text) for pfm in priceFoodMeals]
        except:
            return None
        
    def scrapAddress(self):
        try:
            addressItems =  self.html.find_all('div', self.addressClass)
            return [unicodedata.normalize("NFKD",aditem.text) for aditem in addressItems]
        except:
            return None
        
    def scrapComments(self):
        try:
            comments = self.html.find_all('div', self.commentsClass)
        except:
            return None
        return [re.sub('\n','', comment.text) for comment in comments]
    
    def scrapReviewIdiomsAndClassification(self):
        try:
            reviewIdiomsAndClassification = self.html.find_all('div', self.reviewsIdiomsAndClassificationClass)
        except:
            return None, None
        return reviewIdiomsAndClassification[3].text, reviewIdiomsAndClassification[0].text
        
    def scrapDatesOfVisitsInReview(self):
        try:
            datesReviews = self.html.find_all('div', self.datesOfReviewsClass)
        except:
            return None
        return [dateRev.text for dateRev in datesReviews]
    
    def scrapBubbleRatingInTrip(self):
        try:
            bubbles = self.html.find_all('div', self.bubbleRatingClass)[0].find_all('span', self.bubbleRatingInnerClass)
            bubblesRating = [int(bubble.span['class'][1].replace("bubble_","").strip())/10 for bubble in bubbles]
        except:
            return None
        return bubblesRating
    
    def scrapTripAdvisorRating(self):
        try:
            return float(unicodedata.normalize("NFKD", self.html.find_all('span', self.tripAdvRatingClass)[0].text).strip().replace(",","."))
        except:
            return None
        
    #def pressHourTab(self):
    #    itinerary_clickable = self.driver.find_element(By.CSS_SELECTOR, self.hourTabSelector)
    #    self.driver.implicitly_wait(self.timePause)
    #    itinerary_clickable.click()
    #    return
            
    def driveToLink(self, url):
        start = time.time()
        self.driver.get(url)
        #self.driver.implicitly_wait(self.timePause/2)
        
        #try:
        #    self.driver.find_element("xpath", self.acceptButton).click()
        #except:
        #    pass
        
        #try:
        #    self.driver.implicitly_wait(self.timePause)
        #    self.pressHourTab()
        #except:
        #    pass
            
        self.html = bs.BeautifulSoup(self.driver.page_source, 'html.parser')
        end = time.time()

    
    def scrapPage(self, url):
                
        self.driveToLink(url)       
            
        
        site, menu = self.scrapLinksMainAndMenuWebPages()
        
        near_hotels , near_restaurants, near_attractions = self.scrapNearExternalLinks()
        try:
            idioms_rating, reviews_rating = self.scrapReviewIdiomsAndClassification()
        except:
            idioms_rating, reviews_rating = None, None
        
        self.AnswerTable[self.scrapName()] = {'url': url, \
                                  'triadv_rank': self.scrapNumber(), \
                                  'number_of_reviews': self.scrapNumberReviews(), \
                                  'label_price' : self.scrapLabelPrice(), \
                                  'e_mail' : self.scrapEmail(), \
                                  'telephone': self.scrapTelephone(), \
                                  'status': self.scrapIfVerified(), \
                                  'site': site, \
                                  'menu': menu, \
                                  'near_hotels': near_hotels, \
                                  'near_restaurants': near_restaurants, \
                                  'near_attractions' : near_attractions, \
                                  #'horarios': self.scrapHorarios(), \
                                  'price/food/meals/description' : self.scrapPriceAndFoodAndMeals(), \
                                  'address' : self.scrapAddress(), \
                                  'idioms_rating': idioms_rating, \
                                  'reviews_rating': reviews_rating, \
                                  'reviews': self.scrapComments(), \
                                  'dateOfVisitInReview': self.scrapDatesOfVisitsInReview(), \
                                  'bubble_rating':  self.scrapBubbleRatingInTrip(), \
                                  'tripadv_rating': self.scrapTripAdvisorRating()}
        
        
    def scrapAll(self, links):
        start = time.time()
        
        for link in tqdm.tqdm(links):
            try:
                self.driver.implicitly_wait(20)
                self.scrapPage(link)
            except:
                self.exceptions.append(link)
                
        end = time.time()
        print(start-end)
        return 
    
#lista_numeric = [float(text.replace('.',',')) for text in lista_alpha
      
    #def single_link_scrap(self, link):
        
    

In [ ]:
rec_scrap = ScrapLink()

In [ ]:
rec.links

In [ ]:
rec_scrap.scrapAll(list(rec.links.values()))

In [ ]:
len(rec_scrap.AnswerTable)

In [ ]:
df=pd.DataFrame.from_dict(rec_scrap.AnswerTable, orient='index')

In [ ]:
lista_price = list(['$','$$ - $$$','$$$$'])

In [ ]:
df.label_price = df.label_price.map(lambda x: None if x not in lista_price else x)

In [ ]:
df.telephone.value_counts()

In [ ]:
df.shape

In [ ]:
df.to_csv('recife_trip_adv.csv')

In [ ]:
url = 'https://www.tripadvisor.com.br/Restaurants-g303293-Fortaleza_State_of_Ceara.html'
fort = TripAdvisorScrap(url)

In [ ]:
fort.getLinks(140)

In [ ]:
fort.getLinks(50)

In [ ]:
fort.getLinks(20)

In [ ]:
len(fort.links)

In [ ]:
fort_sc = ScrapLink()

In [ ]:
fort_sc.scrapAll(list(fort.links.values())[:2000])

In [ ]:
fort_sc.scrapAll(list(fort.links.values())[2000:])

In [ ]:
fort_test = ScrapLink

In [ ]:
fort_test.scrapAll(list(fort.links.values()))

In [ ]:
df_frt = pd.DataFrame.from_dict(fort_sc.AnswerTable, orient='index')

In [ ]:
lista_price = list(['$','$$ - $$$','$$-$$$', '$$$', '$$$-$$$$', '$$$ - $$$$', '$$$$', '$$$$-$$$$$', '$$$$ - $$$$$', '$$$$$'])
df_frt.label_price = df_frt.label_price.map(lambda x: None if x not in lista_price else x)

In [ ]:
df_frt.to_csv('fortaleza_trip_adv.csv')

In [ ]:
url ='https://www.tripadvisor.com.br/Restaurants-g303216-Maceio_State_of_Alagoas.html'
maceio = TripAdvisorScrap(url)

In [ ]:
maceio.getLinks(140)

In [ ]:
maceio_scrap = ScrapLink()
maceio_scrap.scrapAll(list(maceio.links.values()))

In [ ]:
len(maceio_scrap.AnswerTable)

In [ ]:
df_maceio = pd.DataFrame.from_dict(maceio_scrap.AnswerTable, orient='index')

In [ ]:
lista_price = list(['$','$$ - $$$','$$-$$$', '$$$', '$$$-$$$$', '$$$ - $$$$', '$$$$', '$$$$-$$$$$', '$$$$ - $$$$$', '$$$$$'])
df_maceio.label_price = df_maceio.label_price.map(lambda x: None if x not in lista_price else x)

In [ ]:
df_maceio.to_csv('maceio_trip_adv.csv')

In [ ]:
url = 'https://www.tripadvisor.com.br/Restaurants-g303272-Salvador_State_of_Bahia.html'
salvador = TripAdvisorScrap(url)

In [ ]:
salvador.getLinks(249)

In [ ]:
len(salvador.links)

In [ ]:
salvador_scrapa = ScrapLink()
salvador_scrapa.scrapAll(list(salvador.links.values()))

In [ ]:
len(salvador.links)

In [ ]:
df_salvador = pd.DataFrame.from_dict(salvador_scrapa.AnswerTable, orient='index')

In [ ]:
lista_price = list(['$','$$ - $$$','$$-$$$', '$$$', '$$$-$$$$', '$$$ - $$$$', '$$$$', '$$$$-$$$$$', '$$$$ - $$$$$', '$$$$$'])
df_salvador.label_price = df_salvador.label_price.map(lambda x: None if x not in lista_price else x)

In [ ]:
df_salvador.to_csv('salvador_trip_adv.csv')

In [ ]:
len(salvador_scrapa.AnswerTable)

In [ ]:
url = 'https://www.tripadvisor.com.br/Restaurants-g303428-Joao_Pessoa_State_of_Paraiba.html'
jp = TripAdvisorScrap(url)

In [ ]:
jp.getLinks(200)

In [ ]:
jp_scrapa = ScrapLink()
jp_scrapa.scrapAll(list(jp.links.values()))

In [ ]:
df_jp = pd.DataFrame.from_dict(jp_scrapa.AnswerTable, orient='index')

In [ ]:
lista_price = list(['$','$$ - $$$','$$-$$$', '$$$', '$$$-$$$$', '$$$ - $$$$', '$$$$', '$$$$-$$$$$', '$$$$ - $$$$$', '$$$$$'])
df_jp.label_price = df_jp.label_price.map(lambda x: None if x not in lista_price else x)

In [ ]:
df_jp.to_csv('joao_pessoa_trip_adv.csv')

In [ ]:
url = 'https://www.tripadvisor.com.br/Restaurants-g303518-Natal_State_of_Rio_Grande_do_Norte.html'
natal = TripAdvisorScrap(url)

In [ ]:
natal.getLinks(85)

In [ ]:
len(natal.links)

In [ ]:

aracaju = TripAdvisorScrap('https://www.tripadvisor.com.br/Restaurants-g303638-Aracaju_State_of_Sergipe.html')
sao_luiz = TripAdvisorScrap('https://www.tripadvisor.com.br/Restaurants-g673267-Sao_Luis_State_of_Maranhao.html')

In [ ]:
aracaju.getLinks(51)

In [ ]:
len(aracaju.links)

In [ ]:
sao_luiz.getLinks(58)

In [ ]:
len(sao_luiz.links)

In [ ]:
natal_scrapa = ScrapLink()
natal_scrapa.scrapAll(list(natal.links.values()))

In [ ]:
sao_luiz_scrapa = ScrapLink()
sao_luiz_scrapa.scrapAll(list(sao_luiz.links.values()))

In [ ]:
aracaju_scrapa = ScrapLink()
aracaju_scrapa.scrapAll(list(aracaju.links.values()))

In [ ]:
df_natal = pd.DataFrame.from_dict(natal_scrapa.AnswerTable, orient='index')
lista_price = list(['$','$$ - $$$','$$-$$$', '$$$', '$$$-$$$$', '$$$ - $$$$', '$$$$', '$$$$-$$$$$', '$$$$ - $$$$$', '$$$$$'])
df_natal.label_price = df_natal.label_price.map(lambda x: None if x not in lista_price else x)
df_natal.to_csv('natal_trip_adv.csv')

In [ ]:
df_sao_luiz = pd.DataFrame.from_dict(sao_luiz_scrapa.AnswerTable, orient='index')
lista_price = list(['$','$$ - $$$','$$-$$$', '$$$', '$$$-$$$$', '$$$ - $$$$', '$$$$', '$$$$-$$$$$', '$$$$ - $$$$$', '$$$$$'])
df_sao_luiz.label_price = df_sao_luiz.label_price.map(lambda x: None if x not in lista_price else x)
df_sao_luiz.to_csv('sao_luis_trip_adv.csv')

In [ ]:
df_aracaju = pd.DataFrame.from_dict(aracaju_scrapa.AnswerTable, orient='index')
lista_price = list(['$','$$ - $$$','$$-$$$', '$$$', '$$$-$$$$', '$$$ - $$$$', '$$$$', '$$$$-$$$$$', '$$$$ - $$$$$', '$$$$$'])
df_aracaju.label_price = df_aracaju.label_price.map(lambda x: None if x not in lista_price else x)
df_aracaju.to_csv('aracaju_trip_adv.csv')